In [ ]:
### GFD: 3-MONTH TREASURY BILL RATE FOR LOCAL INTEREST RATE

In [ ]:
### INITIALIZATION

import requests
import pandas as pd
import datetime
import os
import getpass

In [ ]:
### GFD: DEFINING GFD LOGIN FUNCTION

def gfd_auth(username = None, password = None):
    """
    Pulls a GFD API token and stores it as an environmental variable.
    Parameters
        username: GFD-approved email address.
        password: Password for GFD-approved email address.
    """
    if username is None:
        username = getpass.getpass('Please enter your GFD Finaeon username: ')

    if password is None:
        password = getpass.getpass('Please enter your GFD Finaeon password: ')

    url = 'https://api.globalfinancialdata.com/login/'
    parameters = {'username': username, 'password': password}
    resp = requests.post(url, data = parameters)
    #check for unsuccessful API returns
    if resp.status_code != 200:
        raise ValueError('GFD API request failed with HTTP status code %s' % resp.status_code)

    json_content = resp.json()
    os.environ['GFD_API_TOKEN'] = json_content['token'].strip('"')
    print("GFD API token recieved at %s" % str(datetime.datetime.now()))

In [ ]:
### GFD: RUNNING AUTHORIZATION FUNCTION

gfd_auth('kaminski.ihar@tut.by', '1990757229')

In [16]:
### UN COMTRADE

In [17]:
### UN COMTRADE: INITIALIZATION

import pandas as pd
import requests
import time

In [18]:
### COUNTRY ISO CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    import pandas as pd
    
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']]      
    df_result.index = df_result.index.str.upper()

    return df_result

In [19]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def get_market_membership_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source    
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Results output:
    return ser_market_membership

In [20]:
### UN COMTRADE: COUNTRIES DATA EXTRACTION AND MODIFICATION

def get_un_comtrade_country_id(df_country_codes):
    ### Getting UN Comtrade country info from post request:
    str_UNC_countries_set = 'http://comtrade.un.org/data/cache/partnerAreas.json'
    obj_UNC_countries_set = requests.post(str_UNC_countries_set)
    ### Object to dataframe transformation:
    list_UNC_countries = obj_UNC_countries_set.json()['results']
    df_UNC_countries = pd.DataFrame(list_UNC_countries)
    df_UNC_countries.columns = ['UNC ID', 'COUNTRY']
    df_UNC_countries['COUNTRY'] = df_UNC_countries['COUNTRY'].str.upper()
    df_UNC_countries.replace(dict_map_to_replace, inplace = True)
    df_UNC_countries.set_index('COUNTRY', append = False, drop = True, inplace = True)
    df_UNC_country_id = df_UNC_countries.join(df_country_codes, on = 'COUNTRY', how = 'left').dropna(how = 'any').reset_index(drop = True)
    df_UNC_country_id.drop('ISO LONG', axis = 1, inplace = True)
    df_UNC_country_id.columns = ['Comtrade_ID', 'Country']
    ser_UNC_country_id = df_UNC_country_id.set_index('Country').squeeze().sort_index()
    ### Results output:
    return ser_UNC_country_id

In [21]:
### UN COMTRADE: DATA REQUEST EXECUTION

def get_un_comtrade_data(str_rep_country_id, str_par_country_id, int_max_rec = 50000, str_type = 'C', str_freq = 'M', str_classification_system = 'HS', 
                         str_period = 'all', str_trade_flow = 'All', str_classification_code = 'TOTAL'):
    ### Trade flows codification:
    dict_trade_flow = {'All': 'all', 'Import': '1', 'Export': '2', 're-Export': '3', 're-Import': '4'}
    ### URL prefix:
    str_url_base = 'http://comtrade.un.org/api/get?'
    ### Request URL preparation:
    str_url_request = str_url_base
    list_parameters = []
    list_parameters.append('max=' + str(int_max_rec)) # Usage limit
    list_parameters.append('type=' + str_type) # C = Commodities (merchandise trade data) / S = Services (trade in services data)
    list_parameters.append('freq=' + str_freq) # A = Annual, M = Mpnthly
    list_parameters.append('px=' + str_classification_system) # Trade data classification scheme. See list of valid classifications
    list_parameters.append('ps=' + str_period) # Time period
    list_parameters.append('r=' + str_rep_country_id) # Reporter country
    list_parameters.append('p=' + str_par_country_id) # Partner country
    list_parameters.append('rg=' + dict_trade_flow[str_trade_flow]) # Trade direction
    list_parameters.append('cc=' + str_classification_code) # Commodity code
    list_parameters.append('fmt=json')        
    str_url_request += '&'.join(list_parameters)
    ### Getting UN Comtrade data from post request:    
    obj_unc_dataset = requests.post(str_url_request)
    ### Object to dataframe transformation:    
    list_unc_dataset = obj_unc_dataset.json()['dataset']
    if (len(list_unc_dataset) > 0):
        df_unc_dataset = pd.DataFrame(list_unc_dataset)[['period', 'rtCode', 'ptCode', 'TradeValue']]
        df_unc_dataset.columns = ['Period', 'Reporter_ID', 'Partner_ID', 'Value']    
        df_unc_dataset['Date'] = pd.to_datetime(df_unc_dataset['Period'], format = '%Y%m') + pd.offsets.BMonthEnd()    
        df_unc_dataset = df_unc_dataset[['Date', 'Reporter_ID', 'Partner_ID', 'Value']]
    else:
        df_unc_dataset = pd.DataFrame(columns = ['Date', 'Reporter_ID', 'Partner_ID', 'Value'])
    
    return df_unc_dataset

In [22]:
### UN COMTRADE: GENERAL DATA PREPARATION

### Constants:
int_seconds_to_sleep = 35
int_unc_limit = 5
All = slice(None)
str_path_unc_dataset = 'Data_Files/Source_Files/unc_dataset.h5'
str_unc_exp_total_dataset = 'export_total_dataset'
str_unc_imp_total_dataset = 'import_total_dataset'
### UN Comtrade country names to rename:
dict_map_to_replace = {'BOLIVIA (PLURINATIONAL STATE OF)': 'BOLIVIA',
                       'BOSNIA HERZEGOVINA': 'BOSNIA AND HERZEGOVINA',
                       'BR. INDIAN OCEAN TERR.': 'BRITISH INDIAN OCEAN TERRITORY',
                       'BR. VIRGIN ISDS': 'BRITISH VIRGIN ISLANDS',
                       'BRUNEI DARUSSALAM': 'BRUNEI',
                       'CABO VERDE': 'CAPE VERDE',
                       'CAYMAN ISDS': 'CAYMAN ISLANDS',
                       'CENTRAL AFRICAN REP.': 'CENTRAL AFRICAN REPUBLIC',
                       'CHRISTMAS ISDS': 'CHRISTMAS ISLAND',
                       'COCOS ISDS': 'COCOS ISLANDS',
                       'COOK ISDS': 'COOK ISLANDS',                    
                       'CURAÇAO': 'CURACAO',                          
                       'CZECHIA': 'CZECH REPUBLIC',                    
                       'DEM. REP. OF THE CONGO': 'DEMOCRATIC REPUBLIC OF THE CONGO',                          
                       'DOMINICAN REP.': 'DOMINICAN REPUBLIC',                    
                       'TIMOR-LESTE': 'EAST TIMOR',                          
                       'FALKLAND ISDS (MALVINAS)': 'FALKLAND ISLANDS',                    
                       'FAEROE ISDS': 'FAROE ISLANDS',                                           
                       'CHINA, HONG KONG SAR': 'HONG KONG',                          
                       'CÔTE D\'IVOIRE': 'IVORY COAST',                                           
                       'LAO PEOPLE\'S DEM. REP.': 'LAOS',                                         
                       'CHINA, MACAO SAR': 'MACAU',                          
                       'TFYR OF MACEDONIA': 'MACEDONIA',                    
                       'MARSHALL ISDS': 'MARSHALL ISLANDS',                          
                       'FS MICRONESIA': 'MICRONESIA',                    
                       'REP. OF MOLDOVA': 'MOLDOVA',                          
                       'NETH. ANTILLES': 'NETHERLANDS ANTILLES',                          
                       'DEM. PEOPLE\'S REP. OF KOREA': 'NORTH KOREA',                          
                       'N. MARIANA ISDS': 'NORTHERN MARIANA ISLANDS',                    
                       'STATE OF PALESTINE': 'PALESTINE',                          
                       'CONGO': 'REPUBLIC OF THE CONGO',                          
                       'RÉUNION': 'REUNION',                    
                       'RUSSIAN FEDERATION': 'RUSSIA',                          
                       'SOLOMON ISDS': 'SOLOMON ISLANDS',                    
                       'REP. OF KOREA': 'SOUTH KOREA',                                       
                       'UNITED REP. OF TANZANIA': 'TANZANIA',     
                       'OTHER ASIA, NES': 'TAIWAN',
                       'TURKS AND CAICOS ISDS': 'TURKS AND CAICOS ISLANDS',                    
                       'US VIRGIN ISDS': 'U.S. VIRGIN ISLANDS',                          
                       'USA': 'UNITED STATES',                          
                       'HOLY SEE (VATICAN CITY STATE)': 'VATICAN',                    
                       'VIET NAM': 'VIETNAM',                          
                       'WALLIS AND FUTUNA ISDS': 'WALLIS AND FUTUNA'
                      }
### ISO country codes loading:
df_country_codes = get_country_codes()
### ISON membership loading:
ser_market_membership = get_market_membership_from_excel()
### Getting UN Comtrade country IDs:
ser_UNC_country_id = get_un_comtrade_country_id(df_country_codes)

In [ ]:
### UN COMTRADE: EXPORT DATA EXTRACTION SCRIPT

### Filtering ISON countries only:
ser_UNC_country_id = ser_UNC_country_id.reindex(ser_market_membership.index.get_level_values(1).unique().to_list())
### Concatenation aggregator initializing:
list_dataset = []
### Reporter country looping (5 country groups):
for iter_reporter_group in range((len(ser_UNC_country_id.index) - 1) // int_unc_limit + 1):
    ### Partner country looping (5 country groups):
    for iter_partner_group in range((len(ser_UNC_country_id.index) - 1) // int_unc_limit + 1):
        print(iter_reporter_group * int_unc_limit, '-', (iter_reporter_group + 1) * int_unc_limit - 1, '/', 
              iter_partner_group * int_unc_limit, '-', (iter_partner_group + 1) * int_unc_limit - 1)
#        if (iter_partner_group > 1):
#            break        
        ### Country groups preparing:
        str_reporter_group = ','.join(ser_UNC_country_id.iloc[iter_reporter_group * int_unc_limit : (iter_reporter_group + 1) * int_unc_limit].to_list())
        str_partner_group = ','.join(ser_UNC_country_id.iloc[iter_partner_group * int_unc_limit : (iter_partner_group + 1) * int_unc_limit].to_list())    
        ### Request performing:
        df_iter_dataset = get_un_comtrade_data(str_reporter_group, str_partner_group, str_trade_flow = 'Export')
        list_dataset += [df_iter_dataset]
        ### Pause for API limitations:
        time.sleep(int_seconds_to_sleep)            
#    break
### Results concatenating:
df_loop_dataset = pd.concat(list_dataset, axis = 0, sort = False, ignore_index = True)[['Date', 'Reporter_ID', 'Partner_ID', 'Value']]
df_loop_dataset = df_loop_dataset.astype({'Reporter_ID': 'int16', 'Partner_ID': 'int16', 'Value': 'int64'})
### UN Comtrade country codes replacing to ISON country codes:
df_loop_dataset.loc[All, ['Reporter_ID', 'Partner_ID']] = df_loop_dataset.loc[All, ['Reporter_ID', 'Partner_ID']]\
                                                                         .replace(list(map(int, ser_UNC_country_id.values)), ser_UNC_country_id.index.to_list())
### Series indexing:
ser_unc_dataset = df_loop_dataset.set_index(['Date', 'Reporter_ID', 'Partner_ID'], drop = True).squeeze()
### Data saving:
ser_unc_dataset.to_hdf(path_or_buf = str_path_unc_dataset, key = str_unc_exp_total_dataset, mode = 'w')

In [ ]:
### UN COMTRADE: IMPORT DATA EXTRACTION SCRIPT

### Filtering ISON countries only:
ser_UNC_country_id = ser_UNC_country_id.reindex(ser_market_membership.index.get_level_values(1).unique().to_list())
### Concatenation aggregator initializing:
list_dataset = []
### Reporter country looping (5 country groups):
for iter_reporter_group in range((len(ser_UNC_country_id.index) - 1) // int_unc_limit + 1):
    ### Partner country looping (5 country groups):    
    for iter_partner_group in range((len(ser_UNC_country_id.index) - 1) // int_unc_limit + 1):
        print(iter_reporter_group * int_unc_limit, '-', (iter_reporter_group + 1) * int_unc_limit - 1, '/', 
              iter_partner_group * int_unc_limit, '-', (iter_partner_group + 1) * int_unc_limit - 1)
#        if (iter_partner_group > 1):
#            break        
        ### Country groups preparing:
        str_reporter_group = ','.join(ser_UNC_country_id.iloc[iter_reporter_group * int_unc_limit : (iter_reporter_group + 1) * int_unc_limit].to_list())
        str_partner_group = ','.join(ser_UNC_country_id.iloc[iter_partner_group * int_unc_limit : (iter_partner_group + 1) * int_unc_limit].to_list())    
        ### Request performing:        
        df_iter_dataset = get_un_comtrade_data(str_reporter_group, str_partner_group, str_trade_flow = 'Import')
        list_dataset += [df_iter_dataset]
        ### Pause for API limitations:        
        time.sleep(int_seconds_to_sleep)            
#    break
### Results concatenating:
df_loop_dataset = pd.concat(list_dataset, axis = 0, sort = False, ignore_index = True)[['Date', 'Reporter_ID', 'Partner_ID', 'Value']]
df_loop_dataset = df_loop_dataset.astype({'Reporter_ID': 'int16', 'Partner_ID': 'int16', 'Value': 'int64'})
### UN Comtrade country codes replacing to ISON country codes:
df_loop_dataset.loc[All, ['Reporter_ID', 'Partner_ID']] = df_loop_dataset.loc[All, ['Reporter_ID', 'Partner_ID']]\
                                                                         .replace(list(map(int, ser_UNC_country_id.values)), ser_UNC_country_id.index.to_list())
### Series indexing:
ser_unc_dataset = df_loop_dataset.set_index(['Date', 'Reporter_ID', 'Partner_ID'], drop = True).squeeze()
### Data saving:
ser_unc_dataset.to_hdf(path_or_buf = str_path_unc_dataset, key = str_unc_imp_total_dataset, mode = 'r+')

In [25]:
### UN COMTRADE: TEST: COMPARING WITH ONLINE DATA

ser_unc_total_export = pd.read_hdf(path_or_buf = str_path_unc_dataset, key = str_unc_exp_total_dataset)
ser_unc_total_import = pd.read_hdf(path_or_buf = str_path_unc_dataset, key = str_unc_imp_total_dataset)
print('AT -> BE Export', ser_unc_total_export.loc['2017-04-28', 'AT', 'BE'])
print('AT <- BE Import',ser_unc_total_import.loc['2017-04-28', 'AT', 'BE'])
print('BE -> AT Export', ser_unc_total_export.loc['2017-04-28', 'BE', 'AT'])
print('BE <- AT Import',ser_unc_total_import.loc['2017-04-28', 'BE', 'AT'])

AT -> BE Export 152803671
AT <- BE Import 266718776
BE -> AT Export 304453035
BE <- AT Import 151602472


In [1]:
### BIS: BILATERAL BANK LENDING

In [2]:
### BIS: INITIALIZATION

import pandas as pd
import requests
import zipfile
import io

In [3]:
### BIS: DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def get_market_membership_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source    
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Results output:
    return ser_market_membership

In [4]:
### BIS: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
str_path_bis_csv = 'Data_Files/Source_Files/bis_bank_loans.csv'
str_url_bis_zip = 'https://www.bis.org/statistics/full_bis_lbs_diss_csv.zip'
str_csv_file_name = 'WEBSTATS_LBS_D_PUB_DATAFLOW_csv_col.csv'
str_path_bis_dataset = 'Data_Files/Source_Files/bis_dataset.h5'
str_bis_claim_dataset = 'claim_dataset'
str_bis_liability_dataset = 'claim_dataset'
### ISON membership loading:
ser_market_membership = get_market_membership_from_excel()
### BIS dataset filter:
list_ison_countries = list(map(str, ser_market_membership.index.get_level_values(1).unique()))
str_measure = 'S' # 'Amounts outstanding'
str_claim_position = 'C'
str_liability_position = 'L'
str_instruments = 'A' # All
str_currency_denomination = 'TO1' # All currencies
str_currency_type = 'A' # All currencies
str_parent_country = '5J' # All countries
str_institution = 'A' ### All institutions
str_sector = 'A' ### All sectors
str_position_type = 'N' ### Cross-border
#tup_bis_claim_filter = ('S', 'C', 'A', 'TO1', 'A', '5J', 'A', list_ison_countries, 'A', list_ison_countries, 'N')
#tup_bis_liability_filter = ('S', 'L', 'A', 'TO1', 'A', '5J', 'A', list_ison_countries, 'A', list_ison_countries, 'N')
tup_bis_claim_filter = (str_measure, str_claim_position, str_instruments, str_currency_denomination, str_currency_type, str_parent_country, str_institution,
                        list_ison_countries, str_sector, list_ison_countries, str_position_type)
tup_bis_liability_filter = (str_measure, str_liability_position, str_instruments, str_currency_denomination, str_currency_type, str_parent_country, str_institution,
                            list_ison_countries, str_sector, list_ison_countries, str_position_type)

In [5]:
### BIS: CSV LOADING

### File downloading:
obj_bis_zip = requests.get(str_url_bis_zip)
file_bis_zip = zipfile.ZipFile(io.BytesIO(obj_bis_zip.content))
### Offline alternative:
#df_bis_full_data = pd.read_csv(str_path_bis_csv, index_col = [*range(2, 23, 2)])
### DataFrame creating:
df_bis_full_data = pd.read_csv(file_bis_zip.open(str_csv_file_name), index_col = [*range(2, 23, 2)])

In [6]:
### BIS: DATASET MUNGLING

### Text columns replacing and date columns stacking:
df_bis_full_data.drop(df_bis_full_data.columns[ : 14], axis = 1, inplace = True)
ser_bis_full_data = df_bis_full_data.stack()
### Quarterly date managing:
ser_bis_full_data.index.names = ser_bis_full_data.index.names[ : -1] + ['Date_Q']
ser_bis_full_data.name = 'Value'
### Claims dataset filtering:
ser_bis_claim = ser_bis_full_data.loc[tup_bis_claim_filter].reset_index(level = [0, 1, 2, 3, 4, 5, 6, 8, 10], drop = True)
### Claims resampling to monthly:
df_bis_claim = ser_bis_claim.reset_index('Date_Q')
df_bis_claim.index.names = ['Reporter_ID', 'Partner_ID']
df_bis_claim['Date_Q'] = df_bis_claim['Date_Q'].str.replace('-', '')
df_bis_claim['Date'] = pd.to_datetime(df_bis_claim['Date_Q']) + pd.offsets.BQuarterEnd()
df_bis_claim.drop('Date_Q', axis = 1, inplace = True)
ser_bis_claim = df_bis_claim.set_index('Date', append = True).squeeze().reorder_levels([2, 0, 1])
ser_bis_claim = ser_bis_claim.groupby(['Reporter_ID', 'Partner_ID'])\
                             .apply(lambda iter_group: iter_group.droplevel(['Reporter_ID', 'Partner_ID']).resample('BM').bfill())
### Liabilities dataset filtering:
ser_bis_liability = ser_bis_full_data.loc[tup_bis_liability_filter].reset_index(level = [0, 1, 2, 3, 4, 5, 6, 8, 10], drop = True)
### Liabilities resampling to monthly:
df_bis_liability = ser_bis_liability.reset_index('Date_Q')
df_bis_liability.index.names = ['Reporter_ID', 'Partner_ID']
df_bis_liability['Date_Q'] = df_bis_liability['Date_Q'].str.replace('-', '')
df_bis_liability['Date'] = pd.to_datetime(df_bis_liability['Date_Q']) + pd.offsets.BQuarterEnd()
df_bis_liability.drop('Date_Q', axis = 1, inplace = True)
ser_bis_liability = df_bis_liability.set_index('Date', append = True).squeeze().reorder_levels([2, 0, 1])
ser_bis_liability = ser_bis_liability.groupby(['Reporter_ID', 'Partner_ID'])\
                                     .apply(lambda iter_group: iter_group.droplevel(['Reporter_ID', 'Partner_ID']).resample('BM').bfill())
### Datasets saving
ser_bis_claim.reorder_levels([2, 0, 1]).to_hdf(path_or_buf = str_path_bis_dataset, key = str_bis_claim_dataset, mode = 'w')
ser_bis_liability.reorder_levels([2, 0, 1]).to_hdf(path_or_buf = str_path_bis_dataset, key = str_bis_liability_dataset, mode = 'r+')

In [15]:
### BIS: TEST: RESULTS LOADING

pd.read_hdf(path_or_buf = str_path_bis_dataset, key = str_bis_claim_dataset).tail()

Date        Reporter_ID  Partner_ID
2019-11-29  ZA           ZM            177.0
2019-12-31  ZA           ZM            177.0
2020-01-31  ZA           ZM            137.0
2020-02-28  ZA           ZM            137.0
2020-03-31  ZA           ZM            137.0
Name: Value, dtype: float64

In [ ]:
### BIS: TEST: COLUMNS LEARNING

list_ison = ser_market_membership.index.get_level_values(1).unique().to_list()
print(len([iter_country for iter_country in df_bis_full_data['L_PARENT_CTY'].unique() if (iter_country in list_ison)]))
print(len([iter_country for iter_country in df_bis_full_data['L_REP_CTY'].unique() if (iter_country in list_ison)]))
print(len([iter_country for iter_country in df_bis_full_data['L_CP_COUNTRY'].unique() if (iter_country in list_ison)]))
print(df_bis_full_data[(df_bis_full_data['L_REP_CTY'] != '5A') & (df_bis_full_data['L_PARENT_CTY'] != '5J') \
                 & (df_bis_full_data['L_REP_CTY'] != df_bis_full_data['L_PARENT_CTY'])][df_bis_full_data.columns[10 : 20]])
#df_bis_full_data[(df_bis_full_data['L_REP_CTY'] == 'US') & (df_bis_full_data['L_CP_COUNTRY'] == 'CA')][df_bis_full_data.columns[ : 23]].head()
print(df_bis_full_data['L_DENOM'].unique())

In [1]:
### IMF CPIS: BILATERAL EQUITY & DEBT INVESTMENT POSITIONS

In [2]:
### IMF CPIS: INITIALIZATION

import pandas as pd
import requests
import time

In [3]:
### IMF CPIS: DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def get_market_membership_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source    
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Results output:
    return ser_market_membership

In [14]:
### IMF CPIS: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
dict_request_headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
str_imf_base_url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
str_imf_dataflow_add = 'DataFlow'
str_imf_datastructure_add = 'DataStructure/'
str_imf_codelist_add = 'CodeList/'
str_imf_dataset_add = 'CompactData/'
int_seconds_to_sleep = 3
int_imf_country_limit = 35
str_path_imf_dataset = 'Data_Files/Source_Files/cpis_dataset.h5'
str_asset_imf_dataset = 'cpis_asset_dataset'
str_liability_imf_dataset = 'cpis_liability_dataset'

In [5]:
### IMF CPIS: REQUESTS SESSION INITIALIZING

request_session = requests.Session()
### For avoiding data request errors from IMF Data Service:
request_session.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

In [6]:
### IMF CPIS: DATAFLOW SEARCHING

obj_imf_dataflow_list = request_session.get(str_imf_base_url + str_imf_dataflow_add).json()
df_imf_dataflow = pd.DataFrame(obj_imf_dataflow_list['Structure']['Dataflows']['Dataflow'])
df_imf_dataflow = df_imf_dataflow.assign(Description = df_imf_dataflow['Name'].apply(pd.Series)['#text'].values)[['@id', 'Description']]
ser_imf_dataflow = df_imf_dataflow.set_index('@id', drop = True).squeeze()
### Searching DataFlow code for further requests:
str_imf_cpis_id = ser_imf_dataflow[ser_imf_dataflow.str.contains('CPIS')].index[0].replace('DS-', '')
print(str_imf_cpis_id)

CPIS


In [7]:
### IMF CPIS: DATASTRUCTURE SEARCHING

obj_imf_cpis_structure = request_session.get(str_imf_base_url + str_imf_datastructure_add + str_imf_cpis_id).json()
df_imf_cpis_params = pd.DataFrame(obj_imf_cpis_structure['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension'])\
                                [['@conceptRef', '@codelist', '@isFrequencyDimension']]
### Receiving DataFlow parameters and code lists for each of them:
print(df_imf_cpis_params)

          @conceptRef          @codelist @isFrequencyDimension
0                FREQ            CL_FREQ                  true
1            REF_AREA       CL_AREA_CPIS                   NaN
2           INDICATOR  CL_INDICATOR_CPIS                   NaN
3          REF_SECTOR     CL_SECTOR_CPIS                   NaN
4  COUNTERPART_SECTOR     CL_SECTOR_CPIS                   NaN
5    COUNTERPART_AREA       CL_AREA_CPIS                   NaN


In [ ]:
### IMF CPIS: CODES DESCRIPTION SEARCHING

for int_counter, str_param_code in enumerate(df_imf_cpis_params['@codelist']):
    if (int_counter >= 0):
        time.sleep(int_seconds_to_sleep)    
        obj_imf_cpis_param = request_session.get(str_imf_base_url + str_imf_codelist_add + str_param_code).json()
        df_imf_cpis_param =  pd.DataFrame(obj_imf_cpis_param['Structure']['CodeLists']['CodeList']['Code'])
        ### Receiving values for each code list:
        df_imf_cpis_param = df_imf_cpis_param.assign(Text = df_imf_cpis_param['Description'].apply(pd.Series)['#text'].values)[['@value', 'Text']]
#        print(int_counter, ':', df_imf_cpis_params.iloc[int_counter, All]['@conceptRef'], ':', str_param_code, ':\n', 
#              df_imf_cpis_param.head(10))
    
str_cpis_freq = 'B'
str_cpis_asset_indicator = 'I_A_T_T_T_BP6_USD' 
str_cpis_liability_indicator = 'I_L_T_T_T_BP6_USD'
str_cpis_ref_sector = 'T'
str_cpis_cp_sector = 'T'
list_ison_countries = list(map(str, get_market_membership_from_excel().index.get_level_values(1).unique()))
# 0: FREQ == 'B' # Semi-annual frequency - they don't have Quaterly or Monthly frequency data
# 1: REF_AREA == '??' # Country
# 2: INDICATOR  == 'I_A_T_T_T_BP6_USD' # Assets, Total Investment, BPM6, US Dollars & I_L_T_T_T_BP6_USD    Liabilities, Total Investment, BPM6, US Dollars
# 3: REF_SECTOR == 'T' # Total Holdings (all sectors)
# 4: COUNTERPART_SECTOR  == 'T' # Total Holdings (all sectors)
# 5: COUNTERPART_AREA == '??' # Country

In [10]:
### IMF CPIS ASSETS: REPORTED TOTAL PORTFOLIO INVESTMENT ASSET DATASET RETRIEVING

list_cpis_bilateral = [] # List of bilateral dataframes for future concatenation
str_cpis_const_url = str_imf_base_url + str_imf_dataset_add + str_imf_cpis_id + '/' # Beggining of request URL
### Looping for reporter country groups:
for int_ison_reporter_part in range(0, - (len(list_ison_countries) // ( - int_imf_country_limit))):
    str_cpis_reporters = '+'.join(list_ison_countries[int_ison_reporter_part * int_imf_country_limit : (int_ison_reporter_part + 1) * int_imf_country_limit])
    ### Looping for partner country groups:
    for int_ison_partner_part in range(0, - (len(list_ison_countries) // ( - int_imf_country_limit))):
        str_cpis_partners = '+'.join(list_ison_countries[int_ison_partner_part * int_imf_country_limit : (int_ison_partner_part + 1) * int_imf_country_limit])    
        ### Generating complete request URL:
        str_cpis_full_url = str_cpis_const_url + \
                            '.'.join([str_cpis_freq, str_cpis_reporters, str_cpis_asset_indicator, str_cpis_ref_sector, str_cpis_ref_sector, str_cpis_partners])
        ### Receiving CPIS dataset from IMF API:
        obj_cpis_set = request_session.get(str_cpis_full_url).json()
        ### Converting each bilateral dataset to dataframe and it's mungling:
        for dict_cpis_pair in obj_cpis_set['CompactData']['DataSet']['Series']:
            if isinstance(dict_cpis_pair['Obs'], list):
                df_cpis_bilateral = pd.DataFrame(dict_cpis_pair['Obs'])
            else:
                df_cpis_bilateral = pd.DataFrame([dict_cpis_pair['Obs']])
            df_cpis_bilateral = df_cpis_bilateral[['@TIME_PERIOD', '@OBS_VALUE']]
            df_cpis_bilateral.columns = ['Date_B', 'Value']
            df_cpis_bilateral = df_cpis_bilateral.assign(Reporter_ID = dict_cpis_pair['@REF_AREA'])
            df_cpis_bilateral = df_cpis_bilateral.assign(Partner_ID = dict_cpis_pair['@COUNTERPART_AREA'])
            list_cpis_bilateral.append(df_cpis_bilateral)   
### Bilateral dataset aggregating:
df_cpis = pd.concat(list_cpis_bilateral, axis = 0, ignore_index = True)
df_cpis = df_cpis[df_cpis['Reporter_ID'] != df_cpis['Partner_ID']]
df_cpis = df_cpis.astype({'Reporter_ID': 'str', 'Partner_ID': 'str', 'Value': 'float32'})
### Dataframe converting to stadartized format series:
df_cpis = pd.concat(list_cpis_bilateral, axis = 0, ignore_index = True)
df_cpis = df_cpis[df_cpis['Reporter_ID'] != df_cpis['Partner_ID']]
df_cpis = df_cpis.astype({'Reporter_ID': 'str', 'Partner_ID': 'str', 'Value': 'float32'})
df_cpis.loc[All, 'Date_B'] = df_cpis['Date_B'].str.replace('-B1', 'Q2').str.replace('-B2', 'Q4')
df_cpis['Date'] = pd.to_datetime(df_cpis['Date_B']) + pd.offsets.BQuarterEnd()
ser_asset_cpis_data = df_cpis.drop('Date_B', axis = 1).set_index(['Date', 'Reporter_ID', 'Partner_ID']).squeeze().sort_index(level = [1, 2, 0])
ser_asset_cpis_data = ser_asset_cpis_data.groupby(['Reporter_ID', 'Partner_ID']).\
                                          apply(lambda iter_group: iter_group.droplevel(['Reporter_ID', 'Partner_ID']).resample('BM').bfill()).reorder_levels([2, 0, 1])
### Series saving:
ser_asset_cpis_data.to_hdf(path_or_buf = str_path_imf_dataset, key = str_asset_imf_dataset, mode = 'w')

In [11]:
### IMF CPIS ASSETS: TEST

pd.read_hdf(path_or_buf = str_path_imf_dataset, key = str_asset_imf_dataset).tail()

Date        Reporter_ID  Partner_ID
2019-02-28  ZA           ZM            32.391674
2019-03-29  ZA           ZM            32.391674
2019-04-30  ZA           ZM            32.391674
2019-05-31  ZA           ZM            32.391674
2019-06-28  ZA           ZM            32.391674
Name: Value, dtype: float32

In [18]:
### IMF CPIS LIABILITIES: REPORTED TOTAL PORTFOLIO INVESTMENT LIABILITY DATASET RETRIEVING

list_cpis_bilateral = [] # List of bilateral dataframes for future concatenation
str_cpis_const_url = str_imf_base_url + str_imf_dataset_add + str_imf_cpis_id + '/' # Beggining of request URL
### Looping for reporter country groups:
for int_ison_reporter_part in range(0, - (len(list_ison_countries) // ( - int_imf_country_limit))):
    str_cpis_reporters = '+'.join(list_ison_countries[int_ison_reporter_part * int_imf_country_limit : (int_ison_reporter_part + 1) * int_imf_country_limit])
    ### Looping for partner country groups:
    for int_ison_partner_part in range(0, - (len(list_ison_countries) // ( - int_imf_country_limit))):
        str_cpis_partners = '+'.join(list_ison_countries[int_ison_partner_part * int_imf_country_limit : (int_ison_partner_part + 1) * int_imf_country_limit])    
        ### Generating complete request URL:
        str_cpis_full_url = str_cpis_const_url + \
                            '.'.join([str_cpis_freq, str_cpis_reporters, str_cpis_liability_indicator, str_cpis_ref_sector, str_cpis_ref_sector, str_cpis_partners])
        ### Receiving CPIS dataset from IMF API:
        obj_cpis_set = request_session.get(str_cpis_full_url).json()
        ### Converting each bilateral dataset to dataframe and it's mungling:
        for dict_cpis_pair in obj_cpis_set['CompactData']['DataSet']['Series']:
            if isinstance(dict_cpis_pair['Obs'], list):
                df_cpis_bilateral = pd.DataFrame(dict_cpis_pair['Obs'])
            else:
                df_cpis_bilateral = pd.DataFrame([dict_cpis_pair['Obs']])
            df_cpis_bilateral = df_cpis_bilateral[['@TIME_PERIOD', '@OBS_VALUE']]
            df_cpis_bilateral.columns = ['Date_B', 'Value']
            df_cpis_bilateral = df_cpis_bilateral.assign(Reporter_ID = dict_cpis_pair['@REF_AREA'])
            df_cpis_bilateral = df_cpis_bilateral.assign(Partner_ID = dict_cpis_pair['@COUNTERPART_AREA'])
            list_cpis_bilateral.append(df_cpis_bilateral)   
### Bilateral dataset aggregating:
df_cpis = pd.concat(list_cpis_bilateral, axis = 0, ignore_index = True)
df_cpis = df_cpis[df_cpis['Reporter_ID'] != df_cpis['Partner_ID']]
df_cpis = df_cpis.astype({'Reporter_ID': 'str', 'Partner_ID': 'str', 'Value': 'float32'})
### Dataframe converting to stadartized format series:
df_cpis = pd.concat(list_cpis_bilateral, axis = 0, ignore_index = True)
df_cpis = df_cpis[df_cpis['Reporter_ID'] != df_cpis['Partner_ID']]
df_cpis = df_cpis.astype({'Reporter_ID': 'str', 'Partner_ID': 'str', 'Value': 'float32'})
df_cpis.loc[All, 'Date_B'] = df_cpis['Date_B'].str.replace('-B1', 'Q2').str.replace('-B2', 'Q4')
df_cpis['Date'] = pd.to_datetime(df_cpis['Date_B']) + pd.offsets.BQuarterEnd()
ser_liability_cpis_data = df_cpis.drop('Date_B', axis = 1).set_index(['Date', 'Reporter_ID', 'Partner_ID']).squeeze().sort_index(level = [1, 2, 0])
ser_liability_cpis_data = ser_liability_cpis_data.groupby(['Reporter_ID', 'Partner_ID']).\
                                          apply(lambda iter_group: iter_group.droplevel(['Reporter_ID', 'Partner_ID']).resample('BM').bfill()).reorder_levels([2, 0, 1])
### Series saving:
ser_liability_cpis_data.to_hdf(path_or_buf = str_path_imf_dataset, key = str_liability_imf_dataset, mode = 'r+')

In [19]:
### IMF CPIS LIABILITIES: TEST

pd.read_hdf(path_or_buf = str_path_imf_dataset, key = str_liability_imf_dataset).tail()

Date        Reporter_ID  Partner_ID
2010-08-31  TH           ZM            0.0
2010-09-30  TH           ZM            0.0
2010-10-29  TH           ZM            0.0
2010-11-30  TH           ZM            0.0
2010-12-31  TH           ZM            0.0
Name: Value, dtype: float32

In [2]:
### OECD FDI: FOREIGN DIRECT INVESTMENT

In [3]:
### OECD FDI: INITIALIZATION

import pandas as pd
import requests
import xml.etree.ElementTree as et

In [4]:
### COUNTRY ISO CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    import pandas as pd
    
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']]      
    df_result.index = df_result.index.str.upper()

    return df_result

In [5]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def get_market_membership_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source    
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Results output:
    return ser_market_membership

In [6]:
### OECD FDI: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
str_oecd_base_url = 'https://stats.oecd.org/sdmx-json/data/'
str_oecd_structure_url = 'https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/'
str_fdi_flow_dataset_add = 'FDI_FLOW_CTRY'
str_fdi_pos_dataset_add = 'FDI_POS_CTRY'
### General data:
df_countries = get_country_codes()
ser_ison_membership = get_market_membership_from_excel()
### Results saving:
str_path_fdi_dataset = 'Data_Files/Source_Files/oecd_dataset.h5'
str_fdi_flow_oecd_dataset = 'fdi_flow_dataset'
str_fdi_pos_oecd_dataset = 'fdi_pos_dataset'

In [7]:
### OECD FDI: REQUESTS SESSION INITIALIZING

request_session = requests.Session()
### For avoiding data request errors:
request_session.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})

In [34]:
### OECD FDI: FDI FLOW STRUCTURE REQUEST

obj_oecd_structure = request_session.get(str_oecd_structure_url + str_fdi_flow_dataset_add)
xml_tree_root = et.fromstring(obj_oecd_structure.content)
dict_concepts = {}
dict_dimensions = {}
dict_codelists = {}
for xml_tree_child in xml_tree_root:
    if xml_tree_child.tag.endswith('Concepts'):
        for xml_tree_grand in xml_tree_child:
            str_concept_id = xml_tree_grand.attrib['id']
            str_concept_name = xml_tree_grand[0].text
            dict_concepts[str_concept_id] = str_concept_name
    if xml_tree_child.tag.endswith('KeyFamilies'):
        for xml_tree_family in xml_tree_child:
            for xml_tree_component in xml_tree_family:
                if xml_tree_component.tag.endswith('Components'):
                    for xml_tree_measure in xml_tree_component:
                        if xml_tree_measure.tag.endswith('Dimension'):
                            str_concept_id = xml_tree_measure.attrib['conceptRef']
                            str_concept_cl_id = xml_tree_measure.attrib['codelist']
                            dict_dimensions[str_concept_id] = str_concept_cl_id
    if xml_tree_child.tag.endswith('CodeLists'):       
        for num_tree_grand, xml_tree_grand in enumerate(xml_tree_child):
            str_codelist_id = xml_tree_grand.attrib['id']
            dict_codelist = {}
            for xml_tree_codelist in xml_tree_grand:                
                if xml_tree_codelist.tag.endswith('Code'):
                    str_code_id = xml_tree_codelist.attrib['value']
                    str_code_value = xml_tree_codelist[0].text
                    dict_codelist[str_code_id] = str_code_value
            dict_codelists[str_codelist_id] = dict_codelist

In [35]:
### OECD FDI: FDI FLOW CONCEPTS DESCRIPTION:

dict_concepts

{'COU': 'Reporting country',
 'MEASURE': 'Currency',
 'MEASURE_PRINCIPLE': 'Measurement principle',
 'FDI_TYPE': 'Type of FDI',
 'TYPE_ENTITY': 'Type of entity',
 'ACCOUNTING_ENTRY': 'Accounting entry',
 'LEVEL_COUNTERPART': 'Level of counterpart',
 'COUNTERPART_AREA': 'Partner country/territory',
 'TIME': 'Year',
 'OBS_VALUE': 'Observation Value',
 'TIME_FORMAT': 'Time Format',
 'OBS_STATUS': 'Observation Status',
 'UNIT': 'Unit',
 'POWERCODE': 'Unit multiplier',
 'REFERENCEPERIOD': 'Reference period'}

In [36]:
### OECD FDI: FDI FLOW CONCEPTS SOURCES: 

dict_dimensions

{'COU': 'CL_FDI_FLOW_CTRY_COU',
 'MEASURE': 'CL_FDI_FLOW_CTRY_MEASURE',
 'MEASURE_PRINCIPLE': 'CL_FDI_FLOW_CTRY_MEASURE_PRINCIPLE',
 'FDI_TYPE': 'CL_FDI_FLOW_CTRY_FDI_TYPE',
 'TYPE_ENTITY': 'CL_FDI_FLOW_CTRY_TYPE_ENTITY',
 'ACCOUNTING_ENTRY': 'CL_FDI_FLOW_CTRY_ACCOUNTING_ENTRY',
 'LEVEL_COUNTERPART': 'CL_FDI_FLOW_CTRY_LEVEL_COUNTERPART',
 'COUNTERPART_AREA': 'CL_FDI_FLOW_CTRY_COUNTERPART_AREA',
 'TIME': 'CL_FDI_FLOW_CTRY_TIME'}

In [37]:
### TEMP

pd.concat([pd.Series(dict_concepts), pd.Series(dict_dimensions)], axis = 1, sort = False).dropna()

,0,1
COU,Reporting country,CL_FDI_FLOW_CTRY_COU
MEASURE,Currency,CL_FDI_FLOW_CTRY_MEASURE
MEASURE_PRINCIPLE,Measurement principle,CL_FDI_FLOW_CTRY_MEASURE_PRINCIPLE
FDI_TYPE,Type of FDI,CL_FDI_FLOW_CTRY_FDI_TYPE
TYPE_ENTITY,Type of entity,CL_FDI_FLOW_CTRY_TYPE_ENTITY
ACCOUNTING_ENTRY,Accounting entry,CL_FDI_FLOW_CTRY_ACCOUNTING_ENTRY
LEVEL_COUNTERPART,Level of counterpart,CL_FDI_FLOW_CTRY_LEVEL_COUNTERPART
COUNTERPART_AREA,Partner country/territory,CL_FDI_FLOW_CTRY_COUNTERPART_AREA
TIME,Year,CL_FDI_FLOW_CTRY_TIME


In [38]:
### OECD FDI: FDI FLOW CONCEPT SOURCE CODELISTS:

dict_codelists['CL_FDI_FLOW_CTRY_LEVEL_COUNTERPART']

{'IMC': 'Immediate counterpart (Immediate investor or immediate host)'}

In [13]:
### OECD FDI: FDI FLOW PARAMETERS PREPARATION: Reporters and partners control and preparation

### ISON Countries collecting:
df_ison_countries = df_countries.set_index('ISO SHORT', append = True).reset_index('COUNTRY', drop = True)
df_ison_countries = df_ison_countries.reindex(ser_ison_membership.index.get_level_values(1).unique().to_list())
ser_ison_countries = df_ison_countries.reset_index().set_index('ISO LONG').squeeze()
### OECD reporters vs ISON members:
ser_oecd_reporters = pd.Series(dict_codelists['CL_FDI_FLOW_CTRY_COU'])
ser_oecd_reporters = ser_oecd_reporters.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_reporters[ser_oecd_reporters.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Reporter country with no ISON match:', iter_iso_long)
### OECD partners vs ISON members:
ser_oecd_partners = pd.Series(dict_codelists['CL_FDI_FLOW_CTRY_COUNTERPART_AREA'])
ser_oecd_partners = ser_oecd_partners.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_partners[ser_oecd_partners.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Partner country with no ISON match:', iter_iso_long)
### ISON countries with no OECD partner match:
print('ISON countries with no OECD partner match:', set(ser_ison_countries.dropna().index) - set(ser_oecd_partners.index))
### Lists preparation:
str_reporters_all = '+'.join(ser_oecd_reporters.dropna().index.to_list())
str_partners_all = '+'.join(ser_oecd_partners.dropna().index.to_list())

ISON countries with no OECD partner match: {'ROU'}


In [14]:
### OECD FDI: FDI FLOW PARAMETERS PREPARATION: Non-country parameters:

### Currency:
str_measure = 'USD'
### Direction:
str_direction = '+'.join(['DI', 'DO'])
### Investment type:
str_fdi_type = 'T_FA_F'
### Residence defining:
str_residence = 'ALL'
### Accounting way:
str_accounting = '+'.join(['NET', 'A', 'L'])
### Level counterpart(???):
str_counterpart = 'IMC'

In [15]:
### OECD FDI: FDI FLOW REQUEST CONSTRUCTING

str_fdi_flow_request_params = '.'.join([str_reporters_all, str_measure, str_direction, str_fdi_type, str_residence, str_accounting, str_counterpart, str_partners_all])
str_fdi_flow_request = str_oecd_base_url + str_fdi_flow_dataset_add + '/' + str_fdi_flow_request_params + '/all?detail=DataOnly'
obj_fdi_flow_dataset = request_session.get(str_fdi_flow_request).json()
#str_fdi_flow_request

In [16]:
### OECD FDI: FDI FLOW INDEX DATA COLLECTING:

### Dates:
list_idx_dates = []
for tup_date in obj_fdi_flow_dataset['structure']['dimensions']['observation'][0]['values']:
    list_idx_dates.append(pd.to_datetime(tup_date['id']) + pd.offsets.BYearEnd())
### Parameters:    
list_idx_library = []
for iter_position in obj_fdi_flow_dataset['structure']['dimensions']['series']:
    list_param_values = []
    for tup_parameter in iter_position['values']:
        list_param_values.append(tup_parameter['id'])            
    list_idx_library.append(list_param_values)
### Result:
list_idx_library.append(list_idx_dates)
### Converting to dictionary for future replacing:
list_idx_dict = []
for iter_list in list_idx_library:
    list_idx_dict.append(dict(zip(map(str, range(len(iter_list))), iter_list)))

In [17]:
### OECD FDI: FDI FLOW DATASET RESAMPLING

dict_datasets_res = {}
dict_datasets_source = obj_fdi_flow_dataset['dataSets'][0]['series']
### Parameters and date indexes integration:
for iter_dataset in dict_datasets_source:
    dict_observations = dict_datasets_source[iter_dataset]['observations']
    for iter_observation in dict_observations:
        str_iter_idx = iter_dataset + ':' + iter_observation
        flo_iter_value = dict_observations[iter_observation][0]
        dict_datasets_res[str_iter_idx] = flo_iter_value

In [18]:
### OECD FDI: FDI FLOW DATASET REINDEXATION

df_fdi_flow_data = pd.Series(dict_datasets_res)
df_fdi_flow_data.index = pd.MultiIndex.from_arrays(zip(*df_fdi_flow_data.index.str.split(':')))
int_levels_number = df_fdi_flow_data.index.nlevels
df_fdi_flow_data = df_fdi_flow_data.reset_index()
### Replacing numbers with parameter values:
for iter_level in range(int_levels_number):
    df_fdi_flow_data['level_' + str(iter_level)].replace(list_idx_dict[iter_level], inplace = True)
    ### Replacing long ISO names with short ISO names:
    if (iter_level == 0):
        df_fdi_flow_data['level_' + str(iter_level)].replace(dict(zip(ser_oecd_reporters.index, ser_oecd_reporters)), inplace = True)
    elif (iter_level == 7):
        df_fdi_flow_data['level_' + str(iter_level)].replace(dict(zip(ser_oecd_partners.index, ser_oecd_partners)), inplace = True)      
    ### Directions renaming:
    elif (iter_level == 2):
        df_fdi_flow_data['level_' + str(iter_level)].replace({'DI': 'Inward', 'DO': 'Outward'}, inplace = True)
    ### Flow types renaming:
    elif (iter_level == 5):
        df_fdi_flow_data['level_' + str(iter_level)].replace({'NET': 'Net', 'A': 'Asset', 'L': 'Liability'}, inplace = True)      

### Indexes defining:
ser_fdi_flow_data = df_fdi_flow_data.drop(['level_3', 'level_4', 'level_6'], axis = 1)\
                    .set_index(['level_8', 'level_0', 'level_7', 'level_1', 'level_2', 'level_5']).squeeze()
ser_fdi_flow_data.index.names = ['Date', 'Reporter_ID', 'Partner_ID', 'Currency', 'Direction', 'Accounting']
ser_fdi_flow_data.sort_index(inplace = True)  
ser_fdi_flow_data.name = 'FDI Flows'

In [39]:
### OECD FDI: FDI POSITION STRUCTURE REQUEST

obj_oecd_structure = request_session.get(str_oecd_structure_url + str_fdi_pos_dataset_add)
xml_tree_root = et.fromstring(obj_oecd_structure.content)
dict_concepts = {}
dict_dimensions = {}
dict_codelists = {}
for xml_tree_child in xml_tree_root:
    if xml_tree_child.tag.endswith('Concepts'):
        for xml_tree_grand in xml_tree_child:
            str_concept_id = xml_tree_grand.attrib['id']
            str_concept_name = xml_tree_grand[0].text
            dict_concepts[str_concept_id] = str_concept_name
    if xml_tree_child.tag.endswith('KeyFamilies'):
        for xml_tree_family in xml_tree_child:
            for xml_tree_component in xml_tree_family:
                if xml_tree_component.tag.endswith('Components'):
                    for xml_tree_measure in xml_tree_component:
                        if xml_tree_measure.tag.endswith('Dimension'):
                            str_concept_id = xml_tree_measure.attrib['conceptRef']
                            str_concept_cl_id = xml_tree_measure.attrib['codelist']
                            dict_dimensions[str_concept_id] = str_concept_cl_id
    if xml_tree_child.tag.endswith('CodeLists'):       
        for num_tree_grand, xml_tree_grand in enumerate(xml_tree_child):
            str_codelist_id = xml_tree_grand.attrib['id']
            dict_codelist = {}
            for xml_tree_codelist in xml_tree_grand:                
                if xml_tree_codelist.tag.endswith('Code'):
                    str_code_id = xml_tree_codelist.attrib['value']
                    str_code_value = xml_tree_codelist[0].text
                    dict_codelist[str_code_id] = str_code_value
            dict_codelists[str_codelist_id] = dict_codelist

In [40]:
### OECD FDI: FDI POSITION CONCEPTS SOURCES: 

dict_dimensions

{'COU': 'CL_FDI_POS_CTRY_COU',
 'MEASURE': 'CL_FDI_POS_CTRY_MEASURE',
 'MEASURE_PRINCIPLE': 'CL_FDI_POS_CTRY_MEASURE_PRINCIPLE',
 'FDI_TYPE': 'CL_FDI_POS_CTRY_FDI_TYPE',
 'TYPE_ENTITY': 'CL_FDI_POS_CTRY_TYPE_ENTITY',
 'ACCOUNTING_ENTRY': 'CL_FDI_POS_CTRY_ACCOUNTING_ENTRY',
 'LEVEL_COUNTERPART': 'CL_FDI_POS_CTRY_LEVEL_COUNTERPART',
 'COUNTERPART_AREA': 'CL_FDI_POS_CTRY_COUNTERPART_AREA',
 'TIME': 'CL_FDI_POS_CTRY_TIME'}

In [41]:
### TEMP

pd.concat([pd.Series(dict_concepts), pd.Series(dict_dimensions)], axis = 1, sort = False).dropna()

,0,1
COU,Reporting country,CL_FDI_POS_CTRY_COU
MEASURE,Currency,CL_FDI_POS_CTRY_MEASURE
MEASURE_PRINCIPLE,Measurement principle,CL_FDI_POS_CTRY_MEASURE_PRINCIPLE
FDI_TYPE,Type of FDI,CL_FDI_POS_CTRY_FDI_TYPE
TYPE_ENTITY,Type of entity,CL_FDI_POS_CTRY_TYPE_ENTITY
ACCOUNTING_ENTRY,Accounting entry,CL_FDI_POS_CTRY_ACCOUNTING_ENTRY
LEVEL_COUNTERPART,Level of counterpart,CL_FDI_POS_CTRY_LEVEL_COUNTERPART
COUNTERPART_AREA,Partner country/territory,CL_FDI_POS_CTRY_COUNTERPART_AREA
TIME,Year,CL_FDI_POS_CTRY_TIME


In [42]:
### OECD FDI: FDI POSITION CONCEPT SOURCE CODELISTS:

dict_codelists['CL_FDI_POS_CTRY_FDI_TYPE']

{'LE_FA_F': 'FDI positions -Total',
 'LE_FA_F5': 'FDI positions - Equity (including reinvestment of earnings)',
 'LE_FA_FL': 'FDI positions - Debt'}

In [22]:
### OECD FDI: FDI POSITION PARAMETERS PREPARATION: Non-country parameters:

### Currency:
str_measure = 'USD'
### Direction:
str_direction = '+'.join(['DI', 'DO'])
### Investment type:
str_fdi_type = 'LE_FA_F'
### Residence defining:
str_residence = 'ALL'
### Accounting way:
str_accounting = '+'.join(['NET', 'A', 'L'])
### Level counterpart(???):
str_counterpart = 'IMC'

In [23]:
### OECD FDI: FDI POSITION PARAMETERS PREPARATION: Reporters and partners control and preparation

### ISON Countries collecting:
df_ison_countries = df_countries.set_index('ISO SHORT', append = True).reset_index('COUNTRY', drop = True)
df_ison_countries = df_ison_countries.reindex(ser_ison_membership.index.get_level_values(1).unique().to_list())
ser_ison_countries = df_ison_countries.reset_index().set_index('ISO LONG').squeeze()
### OECD reporters vs ISON members:
ser_oecd_reporters = pd.Series(dict_codelists['CL_FDI_POS_CTRY_COU'])
ser_oecd_reporters = ser_oecd_reporters.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_reporters[ser_oecd_reporters.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Reporter country with no ISON match:', iter_iso_long)
### OECD partners vs ISON members:
ser_oecd_partners = pd.Series(dict_codelists['CL_FDI_POS_CTRY_COUNTERPART_AREA'])
ser_oecd_partners = ser_oecd_partners.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_partners[ser_oecd_partners.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Partner country with no ISON match:', iter_iso_long)
### ISON countries with no OECD partner match:
print('ISON countries with no OECD partner match:', set(ser_ison_countries.dropna().index) - set(ser_oecd_partners.index))
### Lists preparation:
str_reporters_all = '+'.join(ser_oecd_reporters.dropna().index.to_list())
str_partners_all = '+'.join(ser_oecd_partners.dropna().index.to_list())

ISON countries with no OECD partner match: {'ROU'}


In [ ]:
### OECD FDI: FDI POSITION REQUEST CONSTRUCTING

str_fdi_pos_request_params = '.'.join([str_reporters_all, str_measure, str_direction, str_fdi_type, str_residence, str_accounting, str_counterpart, str_partners_all])
str_fdi_pos_request = str_oecd_base_url + str_fdi_pos_dataset_add + '/' + str_fdi_pos_request_params + '/all?detail=DataOnly'
obj_fdi_pos_dataset = request_session.get(str_fdi_pos_request).json()
#str_fdi_pos_request

In [ ]:
### OECD FDI: FDI POSITION INDEX DATA COLLECTING:

### Dates:
list_idx_dates = []
for tup_date in obj_fdi_pos_dataset['structure']['dimensions']['observation'][0]['values']:
    list_idx_dates.append(pd.to_datetime(tup_date['id']) + pd.offsets.BYearEnd())
### Parameters:    
list_idx_library = []
for iter_position in obj_fdi_pos_dataset['structure']['dimensions']['series']:
    list_param_values = []
    for tup_parameter in iter_position['values']:
        list_param_values.append(tup_parameter['id'])            
    list_idx_library.append(list_param_values)
### Result:
list_idx_library.append(list_idx_dates)
### Converting to dictionary for future replacing:
list_idx_dict = []
for iter_list in list_idx_library:
    list_idx_dict.append(dict(zip(map(str, range(len(iter_list))), iter_list)))

In [ ]:
### OECD FDI: FDI POSITION DATASET RESAMPLING

dict_datasets_res = {}
dict_datasets_source = obj_fdi_pos_dataset['dataSets'][0]['series']
### Parameters and date indexes integration:
for iter_dataset in dict_datasets_source:
    dict_observations = dict_datasets_source[iter_dataset]['observations']
    for iter_observation in dict_observations:
        str_iter_idx = iter_dataset + ':' + iter_observation
        flo_iter_value = dict_observations[iter_observation][0]
        dict_datasets_res[str_iter_idx] = flo_iter_value

In [ ]:
### OECD FDI: FDI POSITION DATASET REINDEXATION

df_fdi_pos_data = pd.Series(dict_datasets_res)
df_fdi_pos_data.index = pd.MultiIndex.from_arrays(zip(*df_fdi_pos_data.index.str.split(':')))
int_levels_number = df_fdi_pos_data.index.nlevels
df_fdi_pos_data = df_fdi_pos_data.reset_index()
### Replacing numbers with parameter values:
for iter_level in range(int_levels_number):
    df_fdi_pos_data['level_' + str(iter_level)].replace(list_idx_dict[iter_level], inplace = True)
    ### Replacing long ISO names with short ISO names:
    if (iter_level == 0):
        df_fdi_pos_data['level_' + str(iter_level)].replace(dict(zip(ser_oecd_reporters.index, ser_oecd_reporters)), inplace = True)
    elif (iter_level == 7):
        df_fdi_pos_data['level_' + str(iter_level)].replace(dict(zip(ser_oecd_partners.index, ser_oecd_partners)), inplace = True)      
    ### Directions renaming:
    elif (iter_level == 2):
        df_fdi_pos_data['level_' + str(iter_level)].replace({'DI': 'Inward', 'DO': 'Outward'}, inplace = True)
    ### Flow types renaming:
    elif (iter_level == 5):
        df_fdi_pos_data['level_' + str(iter_level)].replace({'NET': 'Net', 'A': 'Asset', 'L': 'Liability'}, inplace = True)      

### Indexes defining:
ser_fdi_pos_data = df_fdi_pos_data.drop(['level_3', 'level_4', 'level_6'], axis = 1)\
                    .set_index(['level_8', 'level_0', 'level_7', 'level_1', 'level_2', 'level_5']).squeeze()
ser_fdi_pos_data.index.names = ['Date', 'Reporter_ID', 'Partner_ID', 'Currency', 'Direction', 'Accounting']
ser_fdi_pos_data.sort_index(inplace = True)
ser_fdi_pos_data.name = 'FDI Positions'

In [ ]:
### OECD FDI: FDI RESULTS SAVING

ser_fdi_flow_data.to_hdf(path_or_buf = str_path_fdi_dataset, key = str_fdi_flow_oecd_dataset, mode = 'w')
ser_fdi_pos_data.to_hdf(path_or_buf = str_path_fdi_dataset, key = str_fdi_pos_oecd_dataset, mode = 'r+')

In [ ]:
### OECD FDI: FDI RESULTS READING

pd.read_hdf(path_or_buf = str_path_fdi_dataset, key = str_fdi_pos_oecd_dataset).tail()

In [1]:
### CEPII DISTANCES:

In [2]:
### CEPII DISTANCES: INITIALIZATION

import pandas as pd

In [3]:
### COUNTRY ISO CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    import pandas as pd
    
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']]      
    df_result.index = df_result.index.str.upper()

    return df_result

In [4]:
### DEFINING EXTRACTION UNIVERSE DATA FROM GENERAL MS EXCEL SOURCE

def get_market_membership_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    ### Declaring local constants & variables: 
    path_msci = 'Data_Files/Source_Files/sample_universe.xlsx' ### Path for membership source    
    tab_monthly = 'universe_joined'    
    arr_markets_needed = ['DM', 'FM', 'EM']   
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM'}
    no_slice = slice(None)
    ### Extracting universe data:
    df_universe = pd.read_excel(io = path_msci, sheet_name = tab_monthly, skiprows = [0, 2], header = 0, parse_dates = True, 
                                na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    df_universe = df_universe.loc[no_slice, ['dates', 'region', 'ctry']]
    df_universe.columns = ['Date', 'Market', 'Country']
    df_universe.set_index(['Date', 'Country'], inplace = True)
    ser_universe = df_universe.squeeze()
    ser_universe.sort_index(level = [0, 1], inplace = True)
    ser_universe.replace(dict_markets, inplace = True)
    ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
    ### Results output:
    return ser_market_membership

In [5]:
### CEPII DISTANCES: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
str_path_cepii_source = 'Data_Files/Source_Files/CEPII Distance Data/dist_cepii.xls'
### General data:
df_countries = get_country_codes()
ser_ison_membership = get_market_membership_from_excel()
### Results saving:
str_path_cepii_dataset = 'Data_Files/Source_Files/cepii_dataset.h5'
str_distance_dataset = 'distance_dataset'

In [6]:
### CEPII DISTANCES: DATA EXPORT AND REPACKING

#### Source data export:
df_distance_source = pd.read_excel('Data_Files/Source_Files/CEPII Distance Data/dist_cepii.xls', index_col = [0, 1])
### Long to Short Country ID's converting:
df_distance_data = df_distance_source.join(df_countries.set_index('ISO LONG').squeeze(), on = 'iso_o')
df_distance_data.rename({'ISO SHORT': 'From_ID'}, axis = 1, inplace = True)
df_distance_data = df_distance_data.join(df_countries.set_index('ISO LONG').squeeze(), on = 'iso_d')
df_distance_data.rename({'ISO SHORT': 'To_ID'}, axis = 1, inplace = True)
### ISON countries filtering:
list_ison_countries = ser_ison_membership.index.get_level_values(1).unique()
df_distance_data = df_distance_data.dropna().set_index(['From_ID', 'To_ID']).loc[(list_ison_countries, list_ison_countries), ['dist', 'distcap', 'distw', 'distwces']]
df_distance_data = df_distance_data.astype('float16')
### Result saving:
df_distance_data.to_hdf(path_or_buf = str_path_cepii_dataset, key = str_distance_dataset, mode = 'w')

In [ ]:
### CEPII DISTANCES: DATA READING:

pd.read_hdf(path_or_buf = str_path_cepii_dataset, key = str_distance_dataset)

In [2]:
### BLOOMBERG DATA READING

In [3]:
### BLOOMBERG DATA: INITIALIZATION

import pandas as pd

In [4]:
### BLOOMBERG XCRA GDP: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
### Bloomberg data repository::
str_path_bb_hdf = 'Data_Files/Source_Files/Bloomberg_prepared.h5'
str_key_ret = 'bb_ret'
str_key_mmr = 'bb_mmr'
str_key_fx = 'bb_fx'
str_key_mcap = 'bb_mcap'
str_key_reer = 'bb_reer'
str_key_neer = 'bb_neer'
str_key_xcra = 'bb_xcra'

In [11]:
pd.read_hdf(path_or_buf = str_path_bb_hdf, key = str_key_ret).index.get_level_values(1).unique()

DatetimeIndex(['1992-01-31', '1992-02-28', '1992-03-31', '1992-04-30',
               '1992-05-29', '1992-06-30', '1992-07-31', '1992-08-31',
               '1992-09-30', '1992-10-30',
               ...
               '2019-07-31', '2019-08-30', '2019-09-30', '2019-10-31',
               '2019-11-29', '2019-12-31', '2020-01-31', '2020-02-28',
               '2020-03-31', '2020-04-30'],
              dtype='datetime64[ns]', name='Date', length=340, freq=None)